#### Convert "mean_values" excel page to .csv file then run this script with location of input filed mapped at bottom

In [1]:
import pandas as pd
import re
import os

def process_csv_for_prism(input_csv_path):
    """
    Reads a CSV, reshapes the data by stripping specific suffixes (_001, _) from sample names,
    and saves each metric as a separate sheet in an Excel file ready for GraphPad Prism.
    Output file is automatically named <InputFileName>_Prism.xlsx in the same folder.
    """
    
    # --- 1. Dynamic Output Path Generation ---
    base_path = os.path.splitext(input_csv_path)[0]
    output_excel_path = f"{base_path}_Prism.xlsx"

    # --- 2. Load the dataset ---
    try:
        df = pd.read_csv(input_csv_path)
        print(f"Loaded data from: {input_csv_path}")
    except FileNotFoundError:
        print(f"Error: The file was not found at {input_csv_path}")
        return

    # --- 3. Define the exact column order requested ---
    desired_order = [
        "199i46_WT", 
        "H222P", 
        "R225X", 
        "R541C"
    ]

    # --- 4. Identify metric columns ---
    # Exclusion rules: "ROI", "_orig", and keeping "_std" exclusion for cleaner Prism data
    metric_columns = [
        col for col in df.columns 
        if "ROI" not in col 
        and "_orig" not in col 
        and not col.endswith("_std") 
        and col != "Sample"
    ]

    # Initialize the Excel Writer
    with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:
        
        # Iterate through each metric
        for metric in metric_columns:
            
            # Dictionary to hold data lists for this sheet
            data_map = {key: [] for key in desired_order}
            
            # Iterate through rows in the original CSV
            for index, row in df.iterrows():
                # Get the raw sample name
                raw_name = str(row['Sample'])
                
                # --- CLEANING STEPS ---
                # 1. Remove trailing underscore and optional digits (e.g., "_001", "_")
                # This regex matches an underscore at the end, optionally followed by digits
                # It will NOT match "_WT" because W is not a digit.
                clean_name = re.sub(r'_\d*$', '', raw_name)
                
                # 2. Cleanup whitespace
                clean_name = clean_name.strip()

                # Add data to the correct list if the name matches our columns
                if clean_name in data_map:
                    data_map[clean_name].append(row[metric])
                else:
                    # Optional debugging: check for unmatched rows
                    # print(f"Skipping row: {raw_name} -> {clean_name}")
                    pass
            
            # Convert to DataFrame
            # Use dictionary of Series to handle unequal column lengths
            sheet_df = pd.DataFrame({k: pd.Series(v) for k, v in data_map.items()})

            # Ensure strict column ordering
            sheet_df = sheet_df[desired_order]

            # Write to Excel Sheet (Sheet name limit is 31 chars)
            sheet_name = metric[:31]
            sheet_df.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Processing complete.")
    print(f"Output saved to: {output_excel_path}")

# --- Main Execution ---
if __name__ == "__main__":
    # Your specific file path
    input_file = r"C:\Users\m254292\OneDrive - Mayo Clinic\LMNA Project\Nikon Live Imaging\APD Imaging\10-24-2025 CRISPR FluoVolt\CRISPR_LMNA_Homo_FluoVolt_Analysis_10242025_Mean Values.csv"
    
    # Run the function
    process_csv_for_prism(input_file)

Loaded data from: C:\Users\m254292\OneDrive - Mayo Clinic\LMNA Project\Nikon Live Imaging\APD Imaging\10-24-2025 CRISPR FluoVolt\CRISPR_LMNA_Homo_FluoVolt_Analysis_10242025_Mean Values.csv
Processing complete.
Output saved to: C:\Users\m254292\OneDrive - Mayo Clinic\LMNA Project\Nikon Live Imaging\APD Imaging\10-24-2025 CRISPR FluoVolt\CRISPR_LMNA_Homo_FluoVolt_Analysis_10242025_Mean Values_Prism.xlsx
